# Hito 9
## Comprimiendo con `zlib`

Para entender mejor cómo es un `chunk`, hemos hecho una ilustración simple:

![estructura-de-un-chunk](https://github.com/RaquelGG/TM/blob/master/otros/transponer1.png?raw=true)
### Método `pack()`

In [ ]:
def pack(self, seq, chunk):

Hemos decidido comprobar por nuestra cuenta la diferencia de eficiencia entre separar los canales de cada frame en python, de forma normal o usando `numpy`, para ello usamos `timeit`, que cronometra únicamente la parte `stmt` y en setup creamos un array aleatorio, todos del mismo tamaño.

In [44]:
import timeit
py = timeit.timeit(stmt='test[::2]', setup='import os;test=os.urandom(2**15)')
np_sliced = timeit.timeit(stmt='test[::2]', setup='import os;import numpy;test=numpy.frombuffer(os.urandom(2**15),dtype="int16")')
np_trasposed = timeit.timeit(stmt='test.transpose()', setup='import os;import numpy;test=numpy.frombuffer(os.urandom(2**15),dtype="int16").reshape(-1, 2)')
print("con python:", py, "s")
print("con numpy slice :", np_sliced, "s")
print("con numpy transpose :", np_trasposed, "s")

con python: 8.919459699995059 s
con numpy slice : 0.1722801000069012 s
con numpy transpose : 0.14907310000126017 s


Para mejorar la comprensión del código, trasponemos la matriz `chunk`, pasará a ser:

![estructura-de-un-chunk](https://github.com/RaquelGG/TM/blob/master/otros/transponer2.png?raw=true)



In [13]:
import numpy as np
chunk = np.array([[[1], [1]],
                  [[2], [2]],
                  [[3], [3]]])
chunk

array([[[1],
        [1]],

       [[2],
        [2]],

       [[3],
        [3]]])

In [14]:
chunk.transpose()

array([[[1, 2, 3],
        [1, 2, 3]]])

De esta manera solo recorreríamos los canales y queda más legible a la hora de comprimir con `zlib.compress()`:

In [19]:
import zlib

compressed_channels = [zlib.compress(np.ascontiguousarray(channel), level=zlib.Z_BEST_COMPRESSION) for channel in chunk.transpose()]

Codificamos los datos en una secuencia de bytes que pueden ser enviadas por UDP, añadiendo, adicionalmente, el tamaño del primer canal comprimido (para luego poder diferenciar los canales)

In [ ]:
pack_format = f"HH{2*len(compressed_channels[0])}s{2*len(compressed_channels[1])}s"

Y por último, empaquetamos los datos:

In [ ]:
return struct.pack(
    pack_format, 
    seq, 
    2*len(compressed_channels[0]), # tamaño del primer canal comprimido
    *compressed_channels, # * es para compressed_channel[0], [1], ... (expande el array)
)

### Método `unpack()`

In [ ]:
def unpack(self, packed_chunk):

Obtenemos el tamaño del primer canal y del segundo, que se calcula con la diferencia de `packed_chunk` y el tamaño del primer canal:

In [ ]:
first_channel_size, = struct.unpack("H", packed_chunk[SEQ_NO_SIZE:2*SEQ_NO_SIZE])
second_channel_size = len(packed_chunk) - first_channel_size - 2*SEQ_NO_SIZE

Una vez sabemos el tamaño de ambos canales, podemos desempaquetarlo por completo

In [ ]:
seq, _, first_channel_bytes, second_channel_bytes = struct.unpack(
    f"HH{first_channel_size}s{second_channel_size}s",
    packed_chunk,
)

Descomprimimos los canales

In [ ]:
first_channel = np.frombuffer(
    zlib.decompress(first_channel_bytes), 
    dtype='int16',
)   
second_channel = np.frombuffer(
    zlib.decompress(second_channel_bytes),
    dtype='int16'
)

Y por último, volvemos a dejar el chunk con su forma original:

In [ ]:
np.ascontiguousarray(np.concatenate((first_channel, second_channel)).reshape(2,-1).transpose())

![estructura-de-un-chunk](https://github.com/RaquelGG/TM/blob/master/otros/transponer1.png?raw=true)



## Pruebas
En el método `pack()` hemos calculado la tasa de compresión:

In [ ]:
print("size", size, "bytes, compression rate", "{:.2f}%".format(100*(1-size/4096)))
        pack_format = f"HH{2*len(compressed_channels[0])}s{2*len(compressed_channels[1])}s"

In [ ]:
Como resultado:
size 3838 bytes, compression rate 6.30%
size 3926 bytes, compression rate 4.15%
size 3682 bytes, compression rate 10.11%
size 3748 bytes, compression rate 8.50%
size 3818 bytes, compression rate 6.79%
size 3854 bytes, compression rate 5.91%
size 3880 bytes, compression rate 5.27%
size 3852 bytes, compression rate 5.96%
size 3926 bytes, compression rate 4.15%
size 3874 bytes, compression rate 5.42%
size 3706 bytes, compression rate 9.52%

En este caso, la compresión ha mejorado el tamaño de los paquetes, pero hay veces que tiene un resultado contrario al que queremos:

In [ ]:
size 4216 bytes, compression rate -2.93%
size 4170 bytes, compression rate -1.81%
size 4106 bytes, compression rate -0.24%

El tamaño del paquete aumenta.